In [ ]:
import os
from pathlib import Path
notebook_dir = Path.cwd()
layout_dir = notebook_dir.parent.parent / 'layout'
os.chdir(layout_dir)
import gdsfactory as gf
from blocks import *
from comb_drive_tuning import *
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
spring = partial(folded_spring_5um, separation=15, anchor_size=10, flying_bar_height=10,shaft_hole_size=(20,5), shaft_margin=5)
c = spring(length=50, width=2,separation=15)
c = c.extract(['WG'])
c.ports = c.ports
c.write_gds(notebook_dir / 'folded_spring_5um.gds',with_metadata=False)

In [ ]:
import mph
client = mph.start()

In [ ]:
model = client.load(notebook_dir/'folded_spring.mph')
jmodel = model.java

In [ ]:
model.parameter('e1.x', f'{c.ports["e1"].x}[um]')
model.parameter('e1.y', f'{c.ports["e1"].y}[um]')
model.parameter('e2.x', f'{c.ports["e2"].x}[um]')
model.parameter('e2.y', f'{c.ports["e2"].y}[um]')
model.parameter('e3.x', f'{c.ports["e3"].x}[um]')
model.parameter('e3.y', f'{c.ports["e3"].y}[um]')

In [ ]:
imp1 = model/"geometries"/"Geometry 1"/"Import 1"

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.notebook import tqdm
import time
out_dir = notebook_dir / "folded_spring_gds"
out_dir.mkdir(exist_ok=True)
def simulate_stiffness(length, width):
    row = dict(
        spring_length=float(length),
        spring_width=float(width),
        separation=float(15),
        gds_file="",
        xsize=np.nan,
        kxx=np.nan, kzz=np.nan,
        status="fail",
        error=""
    )

    try:
        # --- 1) build parametric layout ---
        c = spring(
            length=length,
            width=width,
            separation=15,
        )
        c_wg = c.extract(['WG'])
        c_wg.ports = c
        xsize = float(c_wg.xsize)
        row["xsize"] = xsize

        # --- 2) write per-case gds + import into COMSOL ---
        # Use a deterministic filename; keep some decimals safe for filenames.
        w_tag = f"{width}".replace(".", "p")
        gds_path = out_dir / f"spring_L{length}_W{w_tag}_S{15}.gds"
        row["gds_file"] = str(gds_path)

        c.write_gds(str(gds_path),with_metadata=False)
        abs_gds_path = Path.absolute(gds_path)
        imp1 = model/"geometries"/"Geometry 1"/"Import 1"
        imp1.property("filename", str(abs_gds_path))
        imp1.property("height",[5,0])
        imp1.property("importlayer",['on','off'])
        imp1.java.importData()

        model.parameter('e1.x', f'{c.ports["e1"].x}[um]')
        model.parameter('e1.y', f'{c.ports["e1"].y}[um]')
        model.parameter('e2.x', f'{c.ports["e2"].x}[um]')
        model.parameter('e2.y', f'{c.ports["e2"].y}[um]')
        model.parameter('e3.x', f'{c.ports["e3"].x}[um]')
        model.parameter('e3.y', f'{c.ports["e3"].y}[um]')

        # --- 4) solve + evaluate stiffness ---
        model.solve()
        kxx, kzz = model.evaluate(['kxx', 'kzz'])[0]

        row["kxx"] = float(kxx)
        row["kzz"] = float(kzz)
        row["status"] = "ok"
        # model.save(str(mph_path))
    except Exception as e:
        row["error"] = f"{type(e).__name__}: {e}"

    return row
simulate_stiffness(40, 1)

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.notebook import tqdm
import time

# =========================
# User settings
# =========================
length_list = list(range(20, 151, 5))   # 20~100 step 5
width_list  = np.linspace(1, 10, 5).round(3).tolist()  # 1~4 共7个点
separation  = 15                                       # fixed for now

out_dir  = Path("gds_sweep")                           # folder for per-case gds files
out_dir.mkdir(parents=True, exist_ok=True)

csv_path = Path("spring_sweep_results.csv")

# selection box half-width in x (your +/-1 logic)
x_box_halfwidth = 1

# =========================
# Helper: append one row to CSV robustly
# =========================
def append_row_to_csv(row_dict, csv_path: Path):
    df_row = pd.DataFrame([row_dict])
    header = not csv_path.exists()
    df_row.to_csv(csv_path, mode="a", header=header, index=False)

# =========================
# Function to simulate stiffness of spring
# =========================
def simulate_stiffness(length, width):
    row = dict(
        spring_length=float(length),
        spring_width=float(width),
        separation=float(15),
        gds_file="",
        xsize=np.nan,
        kxx=np.nan, kzz=np.nan,
        status="fail",
        error=""
    )

    try:
        # --- 1) build parametric layout ---
        c = spring(
            length=length,
            width=width,
            separation=15,
        )
        c_wg = c.extract(['WG'])
        c_wg.ports = c
        xsize = float(c_wg.xsize)
        row["xsize"] = xsize

        # --- 2) write per-case gds + import into COMSOL ---
        # Use a deterministic filename; keep some decimals safe for filenames.
        w_tag = f"{width}".replace(".", "p")
        gds_path = out_dir / f"spring_L{length}_W{w_tag}_S{15}.gds"
        row["gds_file"] = str(gds_path)

        c.write_gds(str(gds_path),with_metadata=False)
        abs_gds_path = Path.absolute(gds_path)
        imp1 = model/"geometries"/"Geometry 1"/"Import 1"
        imp1.property("filename", str(abs_gds_path))
        imp1.property("height",[5,0])
        imp1.property("importlayer",['on','off'])
        imp1.java.importData()

        model.parameter('e1.x', f'{c.ports["e1"].x}[um]')
        model.parameter('e1.y', f'{c.ports["e1"].y}[um]')
        model.parameter('e2.x', f'{c.ports["e2"].x}[um]')
        model.parameter('e2.y', f'{c.ports["e2"].y}[um]')
        model.parameter('e3.x', f'{c.ports["e3"].x}[um]')
        model.parameter('e3.y', f'{c.ports["e3"].y}[um]')

        # --- 4) solve + evaluate stiffness ---
        model.solve()
        kxx, kzz = model.evaluate(['kxx', 'kzz'])[0]

        row["kxx"] = float(kxx)
        row["kzz"] = float(kzz)
        row["status"] = "ok"
        # model.save(str(mph_path))
    except Exception as e:
        row["error"] = f"{type(e).__name__}: {e}"

    return row

# =========================
# Sweep
# =========================
cases = [(L, W) for L in length_list for W in width_list]
t0 = time.time()

for spring_length, spring_width in tqdm(cases, desc="COMSOL sweep", unit="case"):
    row = simulate_stiffness(spring_length, spring_width)

    # --- 5) crash-safe append ---
    append_row_to_csv(row, csv_path)

t1 = time.time()
print(f"Done. CSV saved to: {csv_path.resolve()}")
print(f"GDS saved under: {out_dir.resolve()}")
print(f"Elapsed: {(t1 - t0):.1f} s")